### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [134]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Panda DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset 
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [135]:
# Calculate Variables for Tabulation
lga_schools = len(school_data_complete["School ID"].unique())
lga_students = len(school_data_complete["Student ID"].unique())
lga_budg = school_data_complete["budget"].unique().sum()
lga_math_sc = school_data_complete["maths_score"].sum() / lga_students
lga_read_sc = school_data_complete["reading_score"].sum() / lga_students
lga_math_pass = len(school_data_complete.loc[school_data_complete["maths_score"] >= 50, "Student ID"])\
    / lga_students *100
lga_read_pass = len(school_data_complete.loc[school_data_complete["reading_score"] >= 50, "Student ID"])\
    / lga_students *100
lga_ovall_pass = len(school_data_complete.loc[(school_data_complete["maths_score"] >= 50) & \
                                              (school_data_complete["reading_score"] >= 50), "Student ID"])\
    / lga_students *100

# Select Output Table columns, Label columns and re-format where required
lga_summary_df = pd.DataFrame([{"Total Schools": lga_schools,
                                "Total Students": f'{lga_students:,}',
                                "Total Budget": f'${lga_budg:,.2f}',
                                "Average Maths Score": lga_math_sc,
                                "Average Reading Score": lga_read_sc,
                                "% Passing Maths": lga_math_pass,
                                "% Passing Reading": lga_read_pass,
                                "% Overall Passing": lga_ovall_pass}])

# Display Panda DataFrame
lga_summary_df

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
* % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [136]:
# Flag Student pass in Maths, Set 'okmath' to 1 and store Value to Student Table
passed_math = student_data.loc[student_data["maths_score"] >= 50, 
                               "Student ID"]
student_data["okmath"] = (passed_math + 1) / (school_data_complete["Student ID"] + 1)

# Flag Student pass in Reading, Set 'okread' to 1 and store Value to Student Table
passed_read = student_data.loc[student_data["reading_score"] >= 50, 
                               "Student ID"]
student_data["okread"] = (passed_read + 1) / (school_data_complete["Student ID"] + 1)

# Flag Student pass Overall, Set 'okall' to 1 and store Value to Student Table
passed_all = student_data.loc[(student_data["maths_score"] >= 50) &
                              (student_data["reading_score"] >= 50), 
                              "Student ID"]
student_data["okall"] = (passed_all + 1) / (school_data_complete["Student ID"] + 1)

In [158]:
# Group Scores for Maths, Reading and Overall by School, and Clean 'scores' Table of unnecesary columns
student_scores_data = student_data.groupby(["school_name"]).mean()
del student_scores_data["Student ID"]
del student_scores_data["year"]
del student_scores_data["okmath"]
del student_scores_data["okread"]
del student_scores_data["okall"]

# Group Pass Stats for Maths, Reading and Overall by School and Clean 'pass stats' Table of unnecesary columns
student_pass_data = student_data.groupby(["school_name"]).sum()
del student_pass_data["reading_score"]
del student_pass_data["maths_score"]

# Merge 'school table' with student table containing 'scores'
# Merge that table with table containing 'pass stats'
school_summary_table_pre = pd.merge(school_data.sort_values("school_name"), student_scores_data, 
                    how="left", on=["school_name", "school_name"])
school_summary_table = pd.merge(school_summary_table_pre.sort_values("school_name"), student_pass_data, 
                    how="left", on=["school_name", "school_name"])

# Create DataFrame with school name as index
school_summary_df = pd.DataFrame(school_summary_table)
school_summary_indx_df = school_summary_df.set_index("school_name")

# Calculate school budget per student average--add value to table
studentbudg = school_summary_df["budget"].astype(int) / school_summary_df["size"].astype(int)
school_summary_df["student_budg"] = studentbudg

# Calculate pass rate for maths, reading and overall--add values to table
passmath = (school_summary_df["okmath"].astype(int) / school_summary_df["size"].astype(int)) * 100
school_summary_df["pass_math"] = passmath
passread = (school_summary_df["okread"].astype(int) / school_summary_df["size"].astype(int)) * 100
school_summary_df["pass_read"] = passread
passall = (school_summary_df["okall"].astype(int) / school_summary_df["size"].astype(int)) * 100
school_summary_df["pass_all"] = passall

# Format 'Total School Budget' and 'Per Student Budget' columns
school_summary_df["budget_f"] = school_summary_df["budget"].copy()
school_summary_df["budget_f"]= school_summary_df["budget_f"].map("${:,.2f}".format)
# school_summary_df["student_budg"]= school_summary_df["student_budg"].map("${:,.2f}".format)

# Create Table for use in final Challenge Query re binning
school_summary_org_df = school_summary_df[[
                    "school_name",
                    "type", 
                    "size",
                    "budget_f",
                    "student_budg",
                    "maths_score", 
                    "reading_score",
                    "pass_math",
                    "pass_read",
                    "pass_all"]]

# Label Columns
school_summary_cln_df = school_summary_org_df.rename(columns={
                    "school_name": "School Name",
                    "type": "School Type", 
                    "size": "Total Students", 
                    "budget_f": "Total School Budget",
                    "student_budg": "Per Student Budget",
                    "maths_score": "Average Maths Score",
                    "reading_score": "Average Reading Score",
                    "pass_math": "% Passing Maths",
                    "pass_read": "% Passing Reading",
                    "pass_all": "% Overall Passing"})

# Set DataFrame Index to School Name
school_summary_index_df = school_summary_cln_df.set_index("School Name")

# Display DataFrame
school_summary_index_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",639.0,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",644.0,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",581.0,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",650.0,68.843100,69.039277,82.062592,81.978576,67.191766


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [138]:
top_schools_df = school_summary_index_df.sort_values("% Overall Passing", ascending=False)
top5_schools_df = top_schools_df.iloc[0:5, :]
top5_schools_df

,School Type,Total students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",583.0,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",637.0,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [139]:
bottom_schools_df = school_summary_index_df.sort_values("% Overall Passing")
bottom5_schools_df = bottom_schools_df.iloc[0:5, :]
bottom5_schools_df

,School Type,Total students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",650.0,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",578.0,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",644.0,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [140]:
school_yrs_list = [9, 10, 11, 12]

In [141]:
for x in school_yrs_list:
    school_data_complete.at[school_data_complete["year"] == x, "year" + str(x) + "_math_sc"] = school_data_complete["maths_score"]
    school_data_complete.at[school_data_complete["year"] == x, "year" + str(x) + "_math_num"] = 1

school_group = school_data_complete.groupby(["school_name"]).sum()
school_group_df = pd.DataFrame(school_group)
school_group_df.index.names = ["School Name"]

school_group_df["year9_math_avg"] = school_group_df["year9_math_sc"] / school_group_df["year9_math_num"]
school_group_df["year10_math_avg"] = school_group_df["year10_math_sc"] / school_group_df["year10_math_num"]
school_group_df["year11_math_avg"] = school_group_df["year11_math_sc"] / school_group_df["year11_math_num"]
school_group_df["year12_math_avg"] = school_group_df["year12_math_sc"] / school_group_df["year12_math_num"]

school_group_math_df = school_group_df[["year9_math_avg", "year10_math_avg", "year11_math_avg", "year12_math_avg"]]
school_group_math_df = school_group_math_df.rename(columns={
                            "year9_math_avg": "Year 9",
                            "year10_math_avg": "Year 10",
                            "year11_math_avg": "Year 11",
                            "year12_math_avg": "Year 12"
                            })
school_group_math_df

,Year 9,Year 10,Year 11,Year 12
School Name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


## Reading Score by Year

* Perform the same operations as above for reading scores

In [142]:
for x in school_yrs_list:
    school_data_complete.at[school_data_complete["year"] == x, "year" + str(x) + "_read_sc"] = school_data_complete["reading_score"]
    school_data_complete.at[school_data_complete["year"] == x, "year" + str(x) + "_read_num"] = 1

school_group = school_data_complete.groupby(["school_name"]).sum()
school_group_df = pd.DataFrame(school_group)
school_group_df.index.names = ["School Name"]

school_group_df["year9_read_avg"] = school_group_df["year9_read_sc"] / school_group_df["year9_read_num"]
school_group_df["year10_read_avg"] = school_group_df["year10_read_sc"] / school_group_df["year10_read_num"]
school_group_df["year11_read_avg"] = school_group_df["year11_read_sc"] / school_group_df["year11_read_num"]
school_group_df["year12_read_avg"] = school_group_df["year12_read_sc"] / school_group_df["year12_read_num"]

school_group_read_df = school_group_df[["year9_read_avg", "year10_read_avg", "year11_read_avg", "year12_read_avg"]]
school_group_read_df = school_group_read_df.rename(columns={
                            "year9_read_avg": "Year 9",
                            "year10_read_avg": "Year 10",
                            "year11_read_avg": "Year 11",
                            "year12_read_avg": "Year 12"
                            })
school_group_read_df

,Year 9,Year 10,Year 11,Year 12
School Name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [157]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

school_summary_df["Spending Ranges (Per Student)"] = \
                pd.cut(school_summary_df["student_budg"], \
                        spending_bins, 
                        labels=labels, 
                        include_lowest=True)

spending_maths_scores = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["maths_score"]
spending_reading_scores = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["reading_score"]
spending_passing_maths = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["pass_math"]
spending_passing_reading = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["pass_read"]
overall_passing_spending = school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["pass_all"]

spending_summary = pd.DataFrame({"Average Maths Score": spending_maths_scores,
                                "Average Reading Score": spending_reading_scores,
                                "% Passing Maths": spending_passing_maths,
                                "% Passing Reading": spending_passing_reading,
                                "% Overall Passing": overall_passing_spending
                                })
spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


## Scores by School Size

* Perform the same operations as above, based on school size.

In [160]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary_df["School Size"] = \
                pd.cut(school_summary_df["size"], \
                        size_bins, 
                        labels=labels, 
                        include_lowest=True)

spending_maths_scores = school_summary_df.groupby(["School Size"]).mean()["maths_score"]
spending_reading_scores = school_summary_df.groupby(["School Size"]).mean()["reading_score"]
spending_passing_maths = school_summary_df.groupby(["School Size"]).mean()["pass_math"]
spending_passing_reading = school_summary_df.groupby(["School Size"]).mean()["pass_read"]
overall_passing_spending = school_summary_df.groupby(["School Size"]).mean()["pass_all"]

size_summary = pd.DataFrame({"Average Maths Score": spending_maths_scores,
                                "Average Reading Score": spending_reading_scores,
                                "% Passing Maths": spending_passing_maths,
                                "% Passing Reading": spending_passing_reading,
                                "% Overall Passing": overall_passing_spending
                                })
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

In [161]:
spending_maths_scores = school_summary_df.groupby(["type"]).mean()["maths_score"]
spending_reading_scores = school_summary_df.groupby(["type"]).mean()["reading_score"]
spending_passing_maths = school_summary_df.groupby(["type"]).mean()["pass_math"]
spending_passing_reading = school_summary_df.groupby(["type"]).mean()["pass_read"]
overall_passing_spending = school_summary_df.groupby(["type"]).mean()["pass_all"]

type_summary = pd.DataFrame({"Average Maths Score": spending_maths_scores,
                                "Average Reading Score": spending_reading_scores,
                                "% Passing Maths": spending_passing_maths,
                                "% Passing Reading": spending_passing_reading,
                                "% Overall Passing": overall_passing_spending
                                })
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
